In [1]:
import numpy as np
import pandas as pd
import re
import csv
import spacy
import en_core_web_sm

In [41]:
#df = pd.read_excel('C:/Users/wangs/OneDrive/桌面/Power Status_TVA.xlsx')
df = pd.read_csv('C:/Users/wangs/OneDrive/桌面/Watts Bar.csv')

In [42]:
df

,Date,Unit,Power,Down,Reason or Comment,Change in report (*),Number of Scrams (#)
0,20211208,Watts Bar 1,76,NaN,Holding power,NaN,NaN
1,20211207,Watts Bar 1,76,NaN,Holding power,*,NaN
2,20020714,WATTS BAR 1,0,2002-07-13,TRIP - EN #39058*REACTOR TRIP - EN #39058,NaN,1.0
3,20170529,Watts Bar 1,100,NaN,1 OF 2 161kV LINES OUT OF SERVICE,NaN,NaN
4,20170528,Watts Bar 1,100,NaN,1 OF 2 161kV LINES OUT OF SERVICE,*,NaN
...,...,...,...,...,...,...,...
2806,20220604,Watts Bar 2,0,NaN,NaN,NaN,NaN
2807,20220604,Watts Bar 1,100,NaN,NaN,NaN,NaN
2808,20220603,Watts Bar 2,0,NaN,NaN,NaN,NaN
2809,20220603,Watts Bar 1,100,NaN,NaN,NaN,NaN


In [43]:
for i in range(len(df)):
    df.loc[i, 'Unit'] = str(df.loc[i, 'Unit']).upper()
    df.loc[i, 'Reason or Comment'] = str(df.loc[i, 'Reason or Comment']).upper()

In [64]:
data1 = df[df.Unit == 'WATTS BAR 2']

In [65]:
data1

,Date,Unit,Power,Down,Reason or Comment,Change in report (*),Number of Scrams (#)
5,20161215,WATTS BAR 2,100,NaN,100 MVAR INCOMING LIMIT,NaN,NaN
7,20161214,WATTS BAR 2,100,NaN,100 MVAR INCOMING LIMIT,NaN,NaN
9,20161213,WATTS BAR 2,100,NaN,100 MVAR INCOMING LIMIT,*,NaN
20,20180626,WATTS BAR 2,0,2018-06-22,AUTOMATIC RX TRIP. SEE EN #53467,NaN,NaN
21,20180625,WATTS BAR 2,0,2018-06-22,AUTOMATIC RX TRIP. SEE EN #53467,NaN,NaN
...,...,...,...,...,...,...,...
2802,20220606,WATTS BAR 2,0,NaN,NAN,NaN,NaN
2804,20220605,WATTS BAR 2,0,NaN,NAN,NaN,NaN
2806,20220604,WATTS BAR 2,0,NaN,NAN,NaN,NaN
2808,20220603,WATTS BAR 2,0,NaN,NAN,NaN,NaN


In [66]:
group1 = data1.groupby('Reason or Comment')

In [67]:
len([x for x in group1])

132

In [68]:
state = [x for x in group1][0][1]

In [69]:
state

,Date,Unit,Power,Down,Reason or Comment,Change in report (*),Number of Scrams (#)
5,20161215,WATTS BAR 2,100,NaN,100 MVAR INCOMING LIMIT,NaN,NaN
7,20161214,WATTS BAR 2,100,NaN,100 MVAR INCOMING LIMIT,NaN,NaN
9,20161213,WATTS BAR 2,100,NaN,100 MVAR INCOMING LIMIT,*,NaN


In [70]:
state.dtypes

Date                      int64
Unit                     object
Power                     int64
Down                     object
Reason or Comment        object
Change in report (*)     object
Number of Scrams (#)    float64
dtype: object

In [71]:
state = state.reset_index(drop=True)

In [72]:
state.loc[0,'Reason or Comment']

'100 MVAR INCOMING LIMIT'

In [73]:
state.Date

0    20161215
1    20161214
2    20161213
Name: Date, dtype: int64

In [29]:
def string_to_date(string):
    """string to date"""
    from datetime import datetime
    shift_datetime = datetime.strptime(string, '%Y%m%d')  
    shift_date = shift_datetime.strftime("%Y-%m-%d")  
    return shift_date

def convert_date_from_int(time_series: pd.Series):
    """series -> pandas time"""
    time_series = time_series.astype('str').apply(string_to_date)
    return pd.to_datetime(time_series)

In [75]:
series = pd.Series(state.Date)
time_series = convert_date_from_int(series)

In [76]:
time_series

0   2016-12-15
1   2016-12-14
2   2016-12-13
Name: Date, dtype: datetime64[ns]

In [30]:
def divide(time_series):
    res = []
    i = 0
    res.append([0])
    num_list = 0
    while i < len(time_series)-1:
        if int(str(time_series[i] - time_series[i+1])[:2]) <10:
            res[num_list].append(i+1)
        else:
            num_list+=1
            res.append([])
            res[num_list].append(i+1)
        i+=1
    return res

In [78]:
with open("C:/Users/wangs/OneDrive/桌面/WATTS BAR 2.csv", 'w',newline='') as f:
    writer = csv.writer(f)
    writer.writerow(['Unique Issue', 'Consecutive days de-rated', 'Event date(start)', 'Average Power(% of rate)', 'Generation Losses(in Millions)'])
    for i in range(len([x for x in group1])):
        print(i)
        state = [x for x in group1][i][1]
        state = state.reset_index(drop=True)
        Unique_Issue = state.loc[0, 'Reason or Comment']
        series = pd.Series(state.Date)
        time_series = convert_date_from_int(series)
        divide_event = divide(time_series)
        for j in range(len(divide_event)):
            Consecutive_days = len(divide_event[j])
            Event_date = state.Date[divide_event[j][-1]]
            Average_Power = np.mean(state.Power[divide_event[j][0]:divide_event[j][-1]+1])
            Average_Power = round(Average_Power, 2)
            total = 1.5*Consecutive_days
            Generation_Losses = total - total*(Average_Power/100)
            Generation_Losses = round(Generation_Losses, 2)
            writer.writerow([Unique_Issue, Consecutive_days, Event_date, Average_Power, Generation_Losses])

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131


In [79]:
f.close

<function TextIOWrapper.close()>

In [81]:
data1 = pd.read_csv('C:/Users/wangs/OneDrive/桌面/Browns Ferry.csv')
data2 = pd.read_csv('C:/Users/wangs/OneDrive/桌面/Sequoyah.csv')
data3 = pd.read_csv('C:/Users/wangs/OneDrive/桌面/Watts Bar.csv')

In [82]:
frames = [data1, data2, data3]
 
df = pd.concat(frames)

In [4]:
df = pd.read_csv('C:/Users/wangs/OneDrive/桌面/Power Status_TVA_total.csv')

In [5]:
df

,Date,Unit,Power,Down,Reason or Comment,Change in report (*),Number of Scrams (#)
0,20020806,BROWNS FERRY 3,75,NaN,DUE TO ELEVATED RIVER TEMPERATURE*DERATED DUE...,NaN,NaN
1,20050704,Browns Ferry 3,97,NaN,INCREASING POWER AFTER EXERCISING THE CONTROL...,*,NaN
2,20020915,BROWNS FERRY 2,90,NaN,POWER AT THIS LEVEL FOR 30 DAYS AS OF 9/12 DU...,NaN,NaN
3,20020914,BROWNS FERRY 2,90,NaN,POWER AT THIS LEVEL FOR 30 DAYS AS OF 9/12 DU...,NaN,NaN
4,20020804,BROWNS FERRY 2,76,NaN,POWER FOR A CONTROL ROD ADJUSTMENT*HOLDING PO...,NaN,NaN
...,...,...,...,...,...,...,...
12478,20220604,Watts Bar 2,0,NaN,NaN,NaN,NaN
12479,20220604,Watts Bar 1,100,NaN,NaN,NaN,NaN
12480,20220603,Watts Bar 2,0,NaN,NaN,NaN,NaN
12481,20220603,Watts Bar 1,100,NaN,NaN,NaN,NaN


In [6]:
for i in range(len(df)):
    df.loc[i, 'Unit'] = str(df.loc[i, 'Unit']).upper()
    df.loc[i, 'Reason or Comment'] = str(df.loc[i, 'Reason or Comment']).upper()

In [7]:
df

,Date,Unit,Power,Down,Reason or Comment,Change in report (*),Number of Scrams (#)
0,20020806,BROWNS FERRY 3,75,NaN,DUE TO ELEVATED RIVER TEMPERATURE*DERATED DUE...,NaN,NaN
1,20050704,BROWNS FERRY 3,97,NaN,INCREASING POWER AFTER EXERCISING THE CONTROL...,*,NaN
2,20020915,BROWNS FERRY 2,90,NaN,POWER AT THIS LEVEL FOR 30 DAYS AS OF 9/12 DU...,NaN,NaN
3,20020914,BROWNS FERRY 2,90,NaN,POWER AT THIS LEVEL FOR 30 DAYS AS OF 9/12 DU...,NaN,NaN
4,20020804,BROWNS FERRY 2,76,NaN,POWER FOR A CONTROL ROD ADJUSTMENT*HOLDING PO...,NaN,NaN
...,...,...,...,...,...,...,...
12478,20220604,WATTS BAR 2,0,NaN,NAN,NaN,NaN
12479,20220604,WATTS BAR 1,100,NaN,NAN,NaN,NaN
12480,20220603,WATTS BAR 2,0,NaN,NAN,NaN,NaN
12481,20220603,WATTS BAR 1,100,NaN,NAN,NaN,NaN


In [8]:
#Remove punctuation
def remove_pun(text):
    out_text = str(re.sub('[^A-Za-z0-9 ]+', ' ', str(text)))
    out_text = str(out_text.lstrip(' '))
    out_text = str(out_text.rstrip(' '))
    
    return str(re.sub(r'\s\s+', r' ', str(out_text)))

In [9]:
nlp = en_core_web_sm.load() #text normalization (delete **ing...)

In [10]:
df_ = df[df['Reason or Comment'].str.contains('REFUELING')]

In [11]:
df_

,Date,Unit,Power,Down,Reason or Comment,Change in report (*),Number of Scrams (#)
52,20081128,BROWNS FERRY 1,0,10/25/2008,#1 COOLING TOWER TRANSFORMER OOS; EXITING REFU...,*,NaN
178,20070107,BROWNS FERRY 2,98,NaN,1 OF 7 500 KV LINES OOS FOR PLANNED MAINTENANC...,*,NaN
179,20070106,BROWNS FERRY 2,99,NaN,1 OF 7 500 KV LINES OOS FOR PLANNED MAINTENANC...,NaN,NaN
180,20070105,BROWNS FERRY 2,99,NaN,1 OF 7 500 KV LINES OOS FOR PLANNED MAINTENANC...,NaN,NaN
181,20070104,BROWNS FERRY 2,99,NaN,1 OF 7 500 KV LINES OOS FOR PLANNED MAINTENANC...,*,NaN
...,...,...,...,...,...,...,...
12275,20110511,WATTS BAR 1,0,4/3/2011,REFUELING OUTAGE; TRANSMISSION SYSTEM ALERT ON...,NaN,NaN
12276,20110510,WATTS BAR 1,0,4/3/2011,REFUELING OUTAGE; TRANSMISSION SYSTEM ALERT ON...,NaN,NaN
12277,20110509,WATTS BAR 1,0,4/3/2011,REFUELING OUTAGE; TRANSMISSION SYSTEM ALERT ON...,NaN,NaN
12278,20110508,WATTS BAR 1,0,4/3/2011,REFUELING OUTAGE; TRANSMISSION SYSTEM ALERT ON...,NaN,NaN


In [12]:
final = df.loc[df.index.difference(df_.index)]

In [13]:
df_ = df[df['Reason or Comment'].str.contains('DEFUELED')]
final = final.loc[final.index.difference(df_.index)]
final

,Date,Unit,Power,Down,Reason or Comment,Change in report (*),Number of Scrams (#)
0,20020806,BROWNS FERRY 3,75,NaN,DUE TO ELEVATED RIVER TEMPERATURE*DERATED DUE...,NaN,NaN
1,20050704,BROWNS FERRY 3,97,NaN,INCREASING POWER AFTER EXERCISING THE CONTROL...,*,NaN
2,20020915,BROWNS FERRY 2,90,NaN,POWER AT THIS LEVEL FOR 30 DAYS AS OF 9/12 DU...,NaN,NaN
3,20020914,BROWNS FERRY 2,90,NaN,POWER AT THIS LEVEL FOR 30 DAYS AS OF 9/12 DU...,NaN,NaN
4,20020804,BROWNS FERRY 2,76,NaN,POWER FOR A CONTROL ROD ADJUSTMENT*HOLDING PO...,NaN,NaN
...,...,...,...,...,...,...,...
12478,20220604,WATTS BAR 2,0,NaN,NAN,NaN,NaN
12479,20220604,WATTS BAR 1,100,NaN,NAN,NaN,NaN
12480,20220603,WATTS BAR 2,0,NaN,NAN,NaN,NaN
12481,20220603,WATTS BAR 1,100,NaN,NAN,NaN,NaN


In [14]:
df_ = df[df['Reason or Comment'].str.contains('FUELING')]
final = final.loc[final.index.difference(df_.index)]
final

,Date,Unit,Power,Down,Reason or Comment,Change in report (*),Number of Scrams (#)
0,20020806,BROWNS FERRY 3,75,NaN,DUE TO ELEVATED RIVER TEMPERATURE*DERATED DUE...,NaN,NaN
1,20050704,BROWNS FERRY 3,97,NaN,INCREASING POWER AFTER EXERCISING THE CONTROL...,*,NaN
2,20020915,BROWNS FERRY 2,90,NaN,POWER AT THIS LEVEL FOR 30 DAYS AS OF 9/12 DU...,NaN,NaN
3,20020914,BROWNS FERRY 2,90,NaN,POWER AT THIS LEVEL FOR 30 DAYS AS OF 9/12 DU...,NaN,NaN
4,20020804,BROWNS FERRY 2,76,NaN,POWER FOR A CONTROL ROD ADJUSTMENT*HOLDING PO...,NaN,NaN
...,...,...,...,...,...,...,...
12478,20220604,WATTS BAR 2,0,NaN,NAN,NaN,NaN
12479,20220604,WATTS BAR 1,100,NaN,NAN,NaN,NaN
12480,20220603,WATTS BAR 2,0,NaN,NAN,NaN,NaN
12481,20220603,WATTS BAR 1,100,NaN,NAN,NaN,NaN


In [32]:
final.to_csv('C:/Users/wangs/OneDrive/桌面/Power Status_TVA_remain.csv')

In [15]:
df = pd.read_csv('C:/Users/wangs/OneDrive/桌面/Power Status_TVA_remain.csv')

In [34]:
for i in range(len(df)):
    print(i)
    text = df.loc[i, 'Reason or Comment']
    text_ = remove_pun(text)
    doc = nlp(text_)
    text_ = []
    for token in doc:
        text_.append(token.lemma_)
    final_text = (' '.join(text_)).lower()
    df.loc[i, 'Reason_text'] = final_text

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

1871
1872
1873
1874
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060
2061
2062
2063
2064
2065
2066
2067
2068
2069
2070


3558
3559
3560
3561
3562
3563
3564
3565
3566
3567
3568
3569
3570
3571
3572
3573
3574
3575
3576
3577
3578
3579
3580
3581
3582
3583
3584
3585
3586
3587
3588
3589
3590
3591
3592
3593
3594
3595
3596
3597
3598
3599
3600
3601
3602
3603
3604
3605
3606
3607
3608
3609
3610
3611
3612
3613
3614
3615
3616
3617
3618
3619
3620
3621
3622
3623
3624
3625
3626
3627
3628
3629
3630
3631
3632
3633
3634
3635
3636
3637
3638
3639
3640
3641
3642
3643
3644
3645
3646
3647
3648
3649
3650
3651
3652
3653
3654
3655
3656
3657
3658
3659
3660
3661
3662
3663
3664
3665
3666
3667
3668
3669
3670
3671
3672
3673
3674
3675
3676
3677
3678
3679
3680
3681
3682
3683
3684
3685
3686
3687
3688
3689
3690
3691
3692
3693
3694
3695
3696
3697
3698
3699
3700
3701
3702
3703
3704
3705
3706
3707
3708
3709
3710
3711
3712
3713
3714
3715
3716
3717
3718
3719
3720
3721
3722
3723
3724
3725
3726
3727
3728
3729
3730
3731
3732
3733
3734
3735
3736
3737
3738
3739
3740
3741
3742
3743
3744
3745
3746
3747
3748
3749
3750
3751
3752
3753
3754
3755
3756
3757


5239
5240
5241
5242
5243
5244
5245
5246
5247
5248
5249
5250
5251
5252
5253
5254
5255
5256
5257
5258
5259
5260
5261
5262
5263
5264
5265
5266
5267
5268
5269
5270
5271
5272
5273
5274
5275
5276
5277
5278
5279
5280
5281
5282
5283
5284
5285
5286
5287
5288
5289
5290
5291
5292
5293
5294
5295
5296
5297
5298
5299
5300
5301
5302
5303
5304
5305
5306
5307
5308
5309
5310
5311
5312
5313
5314
5315
5316
5317
5318
5319
5320
5321
5322
5323
5324
5325
5326
5327
5328
5329
5330
5331
5332
5333
5334
5335
5336
5337
5338
5339
5340
5341
5342
5343
5344
5345
5346
5347
5348
5349
5350
5351
5352
5353
5354
5355
5356
5357
5358
5359
5360
5361
5362
5363
5364
5365
5366
5367
5368
5369
5370
5371
5372
5373
5374
5375
5376
5377
5378
5379
5380
5381
5382
5383
5384
5385
5386
5387
5388
5389
5390
5391
5392
5393
5394
5395
5396
5397
5398
5399
5400
5401
5402
5403
5404
5405
5406
5407
5408
5409
5410
5411
5412
5413
5414
5415
5416
5417
5418
5419
5420
5421
5422
5423
5424
5425
5426
5427
5428
5429
5430
5431
5432
5433
5434
5435
5436
5437
5438


6947
6948
6949
6950
6951
6952
6953
6954
6955
6956
6957
6958
6959
6960
6961
6962
6963
6964
6965
6966
6967
6968
6969
6970
6971
6972
6973
6974
6975
6976
6977
6978
6979
6980
6981
6982
6983
6984
6985
6986
6987
6988
6989
6990
6991
6992
6993
6994
6995
6996
6997
6998
6999
7000
7001
7002
7003
7004
7005
7006
7007
7008
7009
7010
7011
7012
7013
7014
7015
7016
7017
7018
7019
7020
7021
7022
7023
7024
7025
7026
7027
7028
7029
7030
7031
7032
7033
7034
7035
7036
7037
7038
7039
7040
7041
7042
7043
7044
7045
7046
7047
7048
7049
7050
7051
7052
7053
7054
7055
7056
7057
7058
7059
7060
7061
7062
7063
7064
7065
7066
7067
7068
7069
7070
7071
7072
7073
7074
7075
7076
7077
7078
7079
7080
7081
7082
7083
7084
7085
7086
7087
7088
7089
7090
7091
7092
7093
7094
7095
7096
7097
7098
7099
7100
7101
7102
7103
7104
7105
7106
7107
7108
7109
7110
7111
7112
7113
7114
7115
7116
7117
7118
7119
7120
7121
7122
7123
7124
7125
7126
7127
7128
7129
7130
7131
7132
7133
7134
7135
7136
7137
7138
7139
7140
7141
7142
7143
7144
7145
7146


In [35]:
df

,Unnamed: 0,Date,Unit,Power,Down,Reason or Comment,Change in report (*),Number of Scrams (#),Reason_text
0,0,20020806,BROWNS FERRY 3,75,NaN,DUE TO ELEVATED RIVER TEMPERATURE*DERATED DUE...,NaN,NaN,due to elevated river temperature derated due ...
1,1,20050704,BROWNS FERRY 3,97,NaN,INCREASING POWER AFTER EXERCISING THE CONTROL...,*,NaN,increase power after exercise the control rod ...
2,2,20020915,BROWNS FERRY 2,90,NaN,POWER AT THIS LEVEL FOR 30 DAYS AS OF 9/12 DU...,NaN,NaN,power at this level for 30 day as of 9 12 due ...
3,3,20020914,BROWNS FERRY 2,90,NaN,POWER AT THIS LEVEL FOR 30 DAYS AS OF 9/12 DU...,NaN,NaN,power at this level for 30 day as of 9 12 due ...
4,4,20020804,BROWNS FERRY 2,76,NaN,POWER FOR A CONTROL ROD ADJUSTMENT*HOLDING PO...,NaN,NaN,power for a control rod adjustment hold power ...
...,...,...,...,...,...,...,...,...,...
7421,12478,20220604,WATTS BAR 2,0,NaN,NAN,NaN,NaN,nan
7422,12479,20220604,WATTS BAR 1,100,NaN,NAN,NaN,NaN,nan
7423,12480,20220603,WATTS BAR 2,0,NaN,NAN,NaN,NaN,nan
7424,12481,20220603,WATTS BAR 1,100,NaN,NAN,NaN,NaN,nan


In [36]:
df.to_csv('C:/Users/wangs/OneDrive/桌面/Power Status_TVA_total_new_remain.csv')

In [14]:
df = pd.read_csv('C:/Users/wangs/OneDrive/桌面/Power Status_TVA_total_new_remain.csv')

In [15]:
df

,Date,Unit,Power,Down,Reason or Comment,Change in report (*),Number of Scrams (#),Reason_text
0,20020806,BROWNS FERRY 3,75,NaN,DUE TO ELEVATED RIVER TEMPERATURE*DERATED DUE...,NaN,NaN,due to elevated river temperature derated due ...
1,20050704,BROWNS FERRY 3,97,NaN,INCREASING POWER AFTER EXERCISING THE CONTROL...,*,NaN,increase power after exercise the control rod ...
2,20020915,BROWNS FERRY 2,90,NaN,POWER AT THIS LEVEL FOR 30 DAYS AS OF 9/12 DU...,NaN,NaN,power at this level for 30 day as of 9 12 due ...
3,20020914,BROWNS FERRY 2,90,NaN,POWER AT THIS LEVEL FOR 30 DAYS AS OF 9/12 DU...,NaN,NaN,power at this level for 30 day as of 9 12 due ...
4,20020804,BROWNS FERRY 2,76,NaN,POWER FOR A CONTROL ROD ADJUSTMENT*HOLDING PO...,NaN,NaN,power for a control rod adjustment hold power ...
...,...,...,...,...,...,...,...,...
7421,20220604,WATTS BAR 2,0,NaN,NAN,NaN,NaN,NaN
7422,20220604,WATTS BAR 1,100,NaN,NAN,NaN,NaN,NaN
7423,20220603,WATTS BAR 2,0,NaN,NAN,NaN,NaN,NaN
7424,20220603,WATTS BAR 1,100,NaN,NAN,NaN,NaN,NaN


In [45]:
#df.replace(to_replace=r'^\s*$',value=np.nan,regex=True,inplace=True)

In [48]:
#data = df.dropna(axis=0,subset=['Reason_text'])

In [16]:
group1 = df.groupby('Reason_text')

In [17]:
len([x for x in group1]) #1867 to 1799 to 1586

1586

In [19]:
[x for x in group1][0][1]

,Date,Unit,Power,Down,Reason or Comment,Change in report (*),Number of Scrams (#),Reason_text
56,20070511,BROWNS FERRY 3,100,NaN,0800 ON 5/11/07 TAGOUT OF 500KV BUS 1,NaN,NaN,0800 on 5 11 07 tagout of 500kv bus 1
57,20070511,BROWNS FERRY 2,100,NaN,0800 ON 5/11/07 TAGOUT OF 500KV BUS 1,NaN,NaN,0800 on 5 11 07 tagout of 500kv bus 1
58,20070510,BROWNS FERRY 3,100,NaN,0800 ON 5/11/07 TAGOUT OF 500KV BUS 1,*,NaN,0800 on 5 11 07 tagout of 500kv bus 1
59,20070510,BROWNS FERRY 2,100,NaN,0800 ON 5/11/07 TAGOUT OF 500KV BUS 1,*,NaN,0800 on 5 11 07 tagout of 500kv bus 1


In [20]:
length = []
for i in range(len([x for x in group1])):
    length.append(len([x for x in group1][i][1]))

In [23]:
"""
import heapq
max_num_index_list = map(length.index, heapq.nlargest(20, length))
"""

In [25]:
sorted_id = sorted(range(len(length)), key=lambda k: length[k], reverse=True)

In [31]:
with open("C:/Users/wangs/OneDrive/桌面/All_Issues_new.csv", 'w',newline='') as f:
    writer = csv.writer(f)
    writer.writerow(['Unique Issue', 'Plant', 'Consecutive days de-rated', 'Event date(start)', 'Average Power(% of rate)', 'Generation Losses(in Millions)', 'Reason_text'])
    for i in sorted_id:
        print(i)
        state = [x for x in group1][i][1]
        state = state.reset_index(drop=True)
        Unique_Issue = state.loc[0, 'Reason or Comment']
        Reason_text = state.loc[0, 'Reason_text']
        state1 = state.groupby('Unit')
        for k in range(len([y for y in state1])):
            state2 = [y for y in state1][k][1]
            state2 = state2.reset_index(drop=True)
            Plant = state2.loc[0, 'Unit']
            series = pd.Series(state2.Date)
            time_series = convert_date_from_int(series)
            divide_event = divide(time_series)
            for j in range(len(divide_event)):
                Consecutive_days = len(divide_event[j])
                Event_date = state2.Date[divide_event[j][-1]]
                Average_Power = np.mean(state2.Power[divide_event[j][0]:divide_event[j][-1]+1])
                Average_Power = round(Average_Power, 2)
                total = 1.5*Consecutive_days
                Generation_Losses = total - total*(Average_Power/100)
                Generation_Losses = round(Generation_Losses, 2)
                writer.writerow([Unique_Issue, Plant, Consecutive_days, Event_date, Average_Power, Generation_Losses, Reason_text])

664
227
929
1048
544
509
833
368
1244
788
780
1207
652
975
99
884
1260
86
79
923
54
361
17
20
1111
992
552
952
512
1402
224
1503
202
854
3
358
1550
118
390
423
955
78
802
485
1296
1331
27
229
1019
57
545
894
247
751
181
827
182
1007
1336
221
587
1137
1472
396
537
930
1585
77
832
921
939
83
276
1438
37
265
272
330
713
129
881
1008
1241
26
49
55
225
233
242
481
600
795
928
931
1575
35
58
199
779
1002
1302
1584
18
239
273
335
553
732
816
889
1064
1498
196
234
414
546
567
685
781
792
978
1030
1051
1063
1084
1202
1210
1227
1401
1513
1540
12
96
101
106
152
177
188
226
263
264
430
434
857
880
942
954
1001
1053
1522
1539
1574
46
102
103
107
201
209
218
222
241
302
782
916
984
1193
1240
1325
1326
1460
1558
1580
2
19
29
33
45
53
124
133
146
249
267
307
334
395
420
432
435
456
476
771
800
828
873
892
944
956
976
1052
1054
1092
1215
1309
1327
1387
1409
1410
1419
1492
1534
1562
1571
1572
1582
41
59
108
138
168
195
208
266
319
320
327
385
404
422
462
477
492
493
590
614
644
645
648
658
734
738
769
7

In [32]:
f.close()

In [4]:
issue = pd.read_csv('C:/Users/wangs/OneDrive/桌面/All_Issues_type.csv')

In [5]:
issue

,Unique Issue,Plant,Consecutive days de-rated,Event date(start),Average Power(% of rate),Generation Losses(in Millions),Reason_text,issue_type
0,INCREASING POWER,BROWNS FERRY 1,2,20220125,47.0,1.59,increase power,increase power
1,INCREASING POWER,BROWNS FERRY 1,1,20211213,80.0,0.30,increase power,increase power
2,INCREASING POWER,BROWNS FERRY 1,1,20201109,80.0,0.30,increase power,increase power
3,INCREASING POWER,BROWNS FERRY 1,2,20200731,73.0,0.81,increase power,increase power
4,INCREASING POWER,BROWNS FERRY 1,1,20200705,99.0,0.02,increase power,increase power
...,...,...,...,...,...,...,...,...
2800,WATERBOX CLEANING,BROWNS FERRY 2,1,20130709,41.0,0.89,waterbox cleaning,waterbox cleaning
2801,"WATERBOX CLEANING, ROD PATTERN ADJUSTMENTS",BROWNS FERRY 3,1,20131222,80.0,0.30,waterbox cleaning rod pattern adjustment,waterbox cleaning
2802,WILL BE COMING OFFLINE @ 1500 EDT TO REPAIR A ...,WATTS BAR 1,1,20080517,83.0,0.26,will be come offline 1500 edt to repair a stat...,will be
2803,WILL CONDUCT A TURBINE VALVE FREEDOM TEST TODAY,SEQUOYAH 2,1,20120505,100.0,0.00,will conduct a turbine valve freedom test today,will conduct


In [6]:
group = issue.groupby('issue_type')

In [7]:
len(group)

556

In [69]:
for i in range(len(issue)):
    print(i)
    type_list = issue.loc[i, 'Reason_text'].split(' ')[0:2]
    type_str = ' '.join(type_list)
    issue.loc[i, 'issue_type'] = type_str

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

2417
2418
2419
2420
2421
2422
2423
2424
2425
2426
2427
2428
2429
2430
2431
2432
2433
2434
2435
2436
2437
2438
2439
2440
2441
2442
2443
2444
2445
2446
2447
2448
2449
2450
2451
2452
2453
2454
2455
2456
2457
2458
2459
2460
2461
2462
2463
2464
2465
2466
2467
2468
2469
2470
2471
2472
2473
2474
2475
2476
2477
2478
2479
2480
2481
2482
2483
2484
2485
2486
2487
2488
2489
2490
2491
2492
2493
2494
2495
2496
2497
2498
2499
2500
2501
2502
2503
2504
2505
2506
2507
2508
2509
2510
2511
2512
2513
2514
2515
2516
2517
2518
2519
2520
2521
2522
2523
2524
2525
2526
2527
2528
2529
2530
2531
2532
2533
2534
2535
2536
2537
2538
2539
2540
2541
2542
2543
2544
2545
2546
2547
2548
2549
2550
2551
2552
2553
2554
2555
2556
2557
2558
2559
2560
2561
2562
2563
2564
2565
2566
2567
2568
2569
2570
2571
2572
2573
2574
2575
2576
2577
2578
2579
2580
2581
2582
2583
2584
2585
2586
2587
2588
2589
2590
2591
2592
2593
2594
2595
2596
2597
2598
2599
2600
2601
2602
2603
2604
2605
2606
2607
2608
2609
2610
2611
2612
2613
2614
2615
2616


In [70]:
issue

,Unique Issue,Plant,Consecutive days de-rated,Event date(start),Average Power(% of rate),Generation Losses(in Millions),Reason_text,issue_type
0,INCREASING POWER,BROWNS FERRY 1,2,20220125,47.0,1.59,increase power,increase power
1,INCREASING POWER,BROWNS FERRY 1,1,20211213,80.0,0.30,increase power,increase power
2,INCREASING POWER,BROWNS FERRY 1,1,20201109,80.0,0.30,increase power,increase power
3,INCREASING POWER,BROWNS FERRY 1,2,20200731,73.0,0.81,increase power,increase power
4,INCREASING POWER,BROWNS FERRY 1,1,20200705,99.0,0.02,increase power,increase power
...,...,...,...,...,...,...,...,...
2800,WATERBOX CLEANING,BROWNS FERRY 2,1,20130709,41.0,0.89,waterbox cleaning,waterbox cleaning
2801,"WATERBOX CLEANING, ROD PATTERN ADJUSTMENTS",BROWNS FERRY 3,1,20131222,80.0,0.30,waterbox cleaning rod pattern adjustment,waterbox cleaning
2802,WILL BE COMING OFFLINE @ 1500 EDT TO REPAIR A ...,WATTS BAR 1,1,20080517,83.0,0.26,will be come offline 1500 edt to repair a stat...,will be
2803,WILL CONDUCT A TURBINE VALVE FREEDOM TEST TODAY,SEQUOYAH 2,1,20120505,100.0,0.00,will conduct a turbine valve freedom test today,will conduct


In [8]:
group_ = issue.groupby('issue_type')

In [9]:
len(group_)

556

In [17]:
new = [x for x in group_][0][1]

In [18]:
for i in range(1,556):
    new = pd.concat([new,[x for x in group_][i][1]])

In [19]:
new

,Unique Issue,Plant,Consecutive days de-rated,Event date(start),Average Power(% of rate),Generation Losses(in Millions),Reason_text,issue_type
1286,0800 ON 5/11/07 TAGOUT OF 500KV BUS 1,BROWNS FERRY 2,2,20070510,100.0,0.00,0800 on 5 11 07 tagout of 500kv bus 1,0800 on
1287,0800 ON 5/11/07 TAGOUT OF 500KV BUS 1,BROWNS FERRY 3,2,20070510,100.0,0.00,0800 on 5 11 07 tagout of 500kv bus 1,0800 on
1929,#1 & 4 CIVS CLOSED,BROWNS FERRY 2,1,20020114,94.0,0.09,1 4 civs closed,1 4
1114,1 500 KV (OF 7) AND 1 161 KV (OF 2) TRANSMISSI...,BROWNS FERRY 1,2,20071101,100.0,0.00,1 500 kv of 7 and 1 161 kv of 2 transmission l...,1 500
1115,1 500 KV (OF 7) AND 1 161 KV (OF 2) TRANSMISSI...,BROWNS FERRY 2,2,20071101,100.0,0.00,1 500 kv of 7 and 1 161 kv of 2 transmission l...,1 500
...,...,...,...,...,...,...,...,...
1592,YELLOW GRID CONDITION DUE TO SCHEDULED GRID MA...,SEQUOYAH 1,3,20061119,100.0,0.00,yellow grid condition due to scheduled grid ma...,yellow grid
1926,YELLOW GRID; EAST CLEVELAND 161KV LINE OOS FOR...,SEQUOYAH 1,1,20110920,100.0,0.00,yellow grid east cleveland 161kv line oos for ...,yellow grid
1927,YELLOW GRID; EAST CLEVELAND 161KV LINE OOS FOR...,SEQUOYAH 2,1,20110920,100.0,0.00,yellow grid east cleveland 161kv line oos for ...,yellow grid
1928,YELLOW GRID RISK DUE TO STORM RELATED VOLTAGE ...,SEQUOYAH 2,2,20090413,100.0,0.00,yellow grid risk due to storm relate voltage c...,yellow grid


In [2]:
dict_keywords = pd.read_csv('C:/Users/wangs/OneDrive/桌面/dict.csv')

In [3]:
dict_keywords

,keywords,term
0,"line oos, line",transmission
1,transformer oos,transformer
2,"automatic scram, automatic trip, trip",scram/trip
3,"condenser tube leak, condenser vacuum, condenser",condenser
4,holding power,holding power
5,forced outage,forced outage
6,condenser cleaning,condenser not interesting
7,"conservative operation, conservative operation...",conservative operations uninteresting
8,"decrease power, downpower due to",include but look at words around
9,"grid risk, grid condition, grid status, grid y...",grid issue


In [4]:
filter_out = pd.read_csv('C:/Users/wangs/OneDrive/桌面/filter_out.csv')

In [5]:
filter_out

,filter_out
0,planned maintenance
1,scheduled maintenance
2,cold shutdown
3,core reload
4,completed
5,in progress
6,increase power
7,reduced power for rod
8,reduced power for planned maintenance
9,hold power


In [6]:
data = pd.read_csv('C:/Users/wangs/OneDrive/桌面/Power Status_TVA_issue_type_08-02-2022.csv')

In [7]:
data

,Unique Issue,Plant,Consecutive days de-rated,Event date(start),Average Power(% of rate),Generation Losses(in Millions),Reason_text,first 2 words,Class
0,0800 ON 5/11/07 TAGOUT OF 500KV BUS 1,BROWNS FERRY 2,2,20070510,100.0,0.00,0800 on 5 11 07 tagout of 500kv bus 1,0800 on,NaN
1,0800 ON 5/11/07 TAGOUT OF 500KV BUS 1,BROWNS FERRY 3,2,20070510,100.0,0.00,0800 on 5 11 07 tagout of 500kv bus 1,0800 on,NaN
2,#1 & 4 CIVS CLOSED,BROWNS FERRY 2,1,20020114,94.0,0.09,1 4 civs closed,1 4,close
3,1 500 KV (OF 7) AND 1 161 KV (OF 2) TRANSMISSI...,BROWNS FERRY 1,2,20071101,100.0,0.00,1 500 kv of 7 and 1 161 kv of 2 transmission l...,1 500,maintenance
4,1 500 KV (OF 7) AND 1 161 KV (OF 2) TRANSMISSI...,BROWNS FERRY 2,2,20071101,100.0,0.00,1 500 kv of 7 and 1 161 kv of 2 transmission l...,1 500,maintenance
...,...,...,...,...,...,...,...,...,...
2800,YELLOW GRID CONDITION DUE TO SCHEDULED GRID MA...,SEQUOYAH 1,3,20061119,100.0,0.00,yellow grid condition due to scheduled grid ma...,yellow grid,yellow grid
2801,YELLOW GRID; EAST CLEVELAND 161KV LINE OOS FOR...,SEQUOYAH 1,1,20110920,100.0,0.00,yellow grid east cleveland 161kv line oos for ...,yellow grid,yellow grid
2802,YELLOW GRID; EAST CLEVELAND 161KV LINE OOS FOR...,SEQUOYAH 2,1,20110920,100.0,0.00,yellow grid east cleveland 161kv line oos for ...,yellow grid,yellow grid
2803,YELLOW GRID RISK DUE TO STORM RELATED VOLTAGE ...,SEQUOYAH 2,2,20090413,100.0,0.00,yellow grid risk due to storm relate voltage c...,yellow grid,yellow grid


In [8]:
def load_TW():
    file_dic = 'C:/Users/wangs/OneDrive/桌面/keywords.txt'
    TW = open(file_dic,encoding='utf-8')
    topicwords = [line.strip() for line in TW]
    TW.close()
    return topicwords

In [9]:
for i in range(len(data)):
    issue = data.loc[i, 'Reason_text']
    Topicwords = load_TW()
    OnlyTW = [word for word in Topicwords if word in issue]
    OnlyTW = (",").join(OnlyTW)
    data.loc[i,'TW'] = OnlyTW
    print(i , OnlyTW)      

0 
1 
2 
3 line,lines oos,
4 line,lines oos,
5 line,lines oos,
6 transformer oos,
7 transformer oos,
8 transformer oos,
9 transformer oos,
10 transformer oos,
11 transformer oos,
12 transformer oos,
13 transformer oos,
14 transformer oos,
15 transformer oos,
16 line,lines oos,
17 line,lines oos,
18 line oos,line,lines oos,line oos,
19 line oos,line,lines oos,line oos,
20 line oos,line,lines oos,line oos,
21 line,lines oos,
22 line,lines oos,
23 line,lines oos,
24 line,lines oos,
25 line,lines oos,
26 line,lines oos,
27 line,lines oos,
28 line,lines oos,
29 line,lines oos,
30 line,lines oos,
31 line,lines oos,
32 line,lines oos,
33 line,lines oos,
34 line,lines oos,
35 line,grid status,lines oos,
36 line,grid status,lines oos,
37 line oos,line,lines oos,line oos,
38 line,lines oos,
39 line,161kv,
40 line oos,line,condenser,lines oos,line oos,
41 line oos,line,condenser,lines oos,line oos,
42 line oos,line,lines oos,line oos,
43 line oos,line,lines oos,line oos,
44 line oos,line,lines oo

886 
887 
888 
889 
890 
891 
892 
893 
894 
895 
896 
897 
898 
899 
900 
901 
902 
903 
904 
905 condenser,
906 condenser,
907 
908 
909 
910 
911 
912 
913 
914 
915 
916 
917 
918 line,lines oos,offsite lines oos,
919 
920 condenser vacuum,condenser,
921 condenser vacuum,condenser,
922 
923 
924 
925 
926 line,
927 
928 conservative operation,conservative operations alert,
929 
930 
931 turbine control,
932 
933 
934 
935 
936 
937 
938 
939 
940 
941 line,
942 
943 
944 
945 
946 condenser,
947 
948 
949 conservative operation,conservative operations alert,
950 
951 recirc,
952 trip,recirc,
953 
954 
955 
956 
957 
958 
959 
960 feedwater,
961 
962 
963 
964 
965 line,
966 
967 
968 
969 
970 line,
971 
972 line oos,line,line oos,feedwater,
973 
974 
975 
976 
977 feedwater,
978 
979 
980 
981 
982 
983 
984 
985 recirc pump,recirc,
986 
987 
988 
989 
990 recirc pump,recirc,
991 
992 
993 
994 
995 
996 
997 
998 
999 
1000 line,line outage,
1001 turbine control,
1002 turbine con

1884 
1885 
1886 
1887 
1888 
1889 
1890 
1891 
1892 
1893 
1894 
1895 
1896 
1897 
1898 
1899 
1900 
1901 
1902 
1903 
1904 
1905 
1906 
1907 
1908 
1909 
1910 trip,recirc pump,recirc,
1911 line oos,line,transformer oos,line oos,
1912 fw,
1913 
1914 
1915 
1916 
1917 
1918 
1919 
1920 
1921 
1922 
1923 
1924 
1925 
1926 
1927 
1928 
1929 
1930 
1931 
1932 
1933 
1934 
1935 
1936 
1937 
1938 
1939 
1940 
1941 
1942 
1943 
1944 
1945 
1946 
1947 
1948 
1949 
1950 
1951 
1952 
1953 
1954 
1955 
1956 
1957 
1958 
1959 
1960 
1961 
1962 
1963 
1964 
1965 
1966 
1967 
1968 
1969 
1970 
1971 
1972 
1973 
1974 
1975 
1976 conservative operation,conservative operations alert,
1977 conservative operation,conservative operations alert,
1978 conservative operation,conservative operations alert,
1979 
1980 
1981 
1982 
1983 
1984 
1985 grid condition,
1986 grid condition,
1987 
1988 
1989 conservative operation,
1990 conservative operation,
1991 conservative operation,
1992 
1993 
1994 
1995 
1996

In [10]:
data

,Unique Issue,Plant,Consecutive days de-rated,Event date(start),Average Power(% of rate),Generation Losses(in Millions),Reason_text,first 2 words,Class,TW
0,0800 ON 5/11/07 TAGOUT OF 500KV BUS 1,BROWNS FERRY 2,2,20070510,100.0,0.00,0800 on 5 11 07 tagout of 500kv bus 1,0800 on,NaN,
1,0800 ON 5/11/07 TAGOUT OF 500KV BUS 1,BROWNS FERRY 3,2,20070510,100.0,0.00,0800 on 5 11 07 tagout of 500kv bus 1,0800 on,NaN,
2,#1 & 4 CIVS CLOSED,BROWNS FERRY 2,1,20020114,94.0,0.09,1 4 civs closed,1 4,close,
3,1 500 KV (OF 7) AND 1 161 KV (OF 2) TRANSMISSI...,BROWNS FERRY 1,2,20071101,100.0,0.00,1 500 kv of 7 and 1 161 kv of 2 transmission l...,1 500,maintenance,"line,lines oos,"
4,1 500 KV (OF 7) AND 1 161 KV (OF 2) TRANSMISSI...,BROWNS FERRY 2,2,20071101,100.0,0.00,1 500 kv of 7 and 1 161 kv of 2 transmission l...,1 500,maintenance,"line,lines oos,"
...,...,...,...,...,...,...,...,...,...,...
2800,YELLOW GRID CONDITION DUE TO SCHEDULED GRID MA...,SEQUOYAH 1,3,20061119,100.0,0.00,yellow grid condition due to scheduled grid ma...,yellow grid,yellow grid,"grid condition,"
2801,YELLOW GRID; EAST CLEVELAND 161KV LINE OOS FOR...,SEQUOYAH 1,1,20110920,100.0,0.00,yellow grid east cleveland 161kv line oos for ...,yellow grid,yellow grid,"line oos,line,161kv,line oos,"
2802,YELLOW GRID; EAST CLEVELAND 161KV LINE OOS FOR...,SEQUOYAH 2,1,20110920,100.0,0.00,yellow grid east cleveland 161kv line oos for ...,yellow grid,yellow grid,"line oos,line,161kv,line oos,"
2803,YELLOW GRID RISK DUE TO STORM RELATED VOLTAGE ...,SEQUOYAH 2,2,20090413,100.0,0.00,yellow grid risk due to storm relate voltage c...,yellow grid,yellow grid,"grid risk,"


In [33]:
for i in range(len(data)):
    print(i)
    tw = data.loc[i,'TW']
    TW = tw.split(',')
    if TW != ['']:
        for j in range(len(dict_keywords)):
            #print(dict_keywords.loc[j,'keywords'])
            kw = dict_keywords.loc[j,'keywords']
            if TW[0] in kw:
                data.loc[i,'term'] = dict_keywords.loc[j,'term']
                print(dict_keywords.loc[j,'term'])
                break
    else:
        data.loc[i,'term'] = ''

0
1
2
3
transmission 
4
transmission 
5
transmission 
6
transformer
7
transformer
8
transformer
9
transformer
10
transformer
11
transformer
12
transformer
13
transformer
14
transformer
15
transformer
16
transmission 
17
transmission 
18
transmission 
19
transmission 
20
transmission 
21
transmission 
22
transmission 
23
transmission 
24
transmission 
25
transmission 
26
transmission 
27
transmission 
28
transmission 
29
transmission 
30
transmission 
31
transmission 
32
transmission 
33
transmission 
34
transmission 
35
transmission 
36
transmission 
37
transmission 
38
transmission 
39
transmission 
40
transmission 
41
transmission 
42
transmission 
43
transmission 
44
transmission 
45
transmission 
46
transmission 
47
transmission 
48
transmission 
49
transmission 
50
transmission 
51
transmission 
52
transmission 
53
54
55
56
transmission 
57
transmission 
58
transmission 
59
transmission 
60
transmission 
61
transmission 
62
63
64
65
transmission 
66
transmission 
67
transmission 


1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
scram/trip
1911
transmission 
1912
feedwater heaters
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
conservative operations uninteresting
1977
conservative operations uninteresting
1978
conservative operations uninteresting
1979
1980
1981
1982
1983
1984
1985
grid issue
1986
grid issue
1987
1988
1989
conservative operations uninteresting
1990
conservative operations uninteresting
1991
conservative operations uninteresting
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
scram/trip
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
transmission 
2015
2016
2017
2018
2019
2020
2021
2022
transmi

In [34]:
data

,Unique Issue,Plant,Consecutive days de-rated,Event date(start),Average Power(% of rate),Generation Losses(in Millions),Reason_text,first 2 words,Class,TW,term
0,0800 ON 5/11/07 TAGOUT OF 500KV BUS 1,BROWNS FERRY 2,2,20070510,100.0,0.00,0800 on 5 11 07 tagout of 500kv bus 1,0800 on,NaN,,
1,0800 ON 5/11/07 TAGOUT OF 500KV BUS 1,BROWNS FERRY 3,2,20070510,100.0,0.00,0800 on 5 11 07 tagout of 500kv bus 1,0800 on,NaN,,
2,#1 & 4 CIVS CLOSED,BROWNS FERRY 2,1,20020114,94.0,0.09,1 4 civs closed,1 4,close,,
3,1 500 KV (OF 7) AND 1 161 KV (OF 2) TRANSMISSI...,BROWNS FERRY 1,2,20071101,100.0,0.00,1 500 kv of 7 and 1 161 kv of 2 transmission l...,1 500,maintenance,"line,lines oos,",transmission
4,1 500 KV (OF 7) AND 1 161 KV (OF 2) TRANSMISSI...,BROWNS FERRY 2,2,20071101,100.0,0.00,1 500 kv of 7 and 1 161 kv of 2 transmission l...,1 500,maintenance,"line,lines oos,",transmission
...,...,...,...,...,...,...,...,...,...,...,...
2800,YELLOW GRID CONDITION DUE TO SCHEDULED GRID MA...,SEQUOYAH 1,3,20061119,100.0,0.00,yellow grid condition due to scheduled grid ma...,yellow grid,yellow grid,"grid condition,",grid issue
2801,YELLOW GRID; EAST CLEVELAND 161KV LINE OOS FOR...,SEQUOYAH 1,1,20110920,100.0,0.00,yellow grid east cleveland 161kv line oos for ...,yellow grid,yellow grid,"line oos,line,161kv,line oos,",transmission
2802,YELLOW GRID; EAST CLEVELAND 161KV LINE OOS FOR...,SEQUOYAH 2,1,20110920,100.0,0.00,yellow grid east cleveland 161kv line oos for ...,yellow grid,yellow grid,"line oos,line,161kv,line oos,",transmission
2803,YELLOW GRID RISK DUE TO STORM RELATED VOLTAGE ...,SEQUOYAH 2,2,20090413,100.0,0.00,yellow grid risk due to storm relate voltage c...,yellow grid,yellow grid,"grid risk,",grid issue


In [38]:
data.replace(to_replace=r'^\s*$',value=np.nan,regex=True,inplace=True)

In [39]:
data1 = data.dropna(subset = ['term'])

In [40]:
data1

,Unique Issue,Plant,Consecutive days de-rated,Event date(start),Average Power(% of rate),Generation Losses(in Millions),Reason_text,first 2 words,Class,TW,term
3,1 500 KV (OF 7) AND 1 161 KV (OF 2) TRANSMISSI...,BROWNS FERRY 1,2,20071101,100.0,0.00,1 500 kv of 7 and 1 161 kv of 2 transmission l...,1 500,maintenance,"line,lines oos,",transmission
4,1 500 KV (OF 7) AND 1 161 KV (OF 2) TRANSMISSI...,BROWNS FERRY 2,2,20071101,100.0,0.00,1 500 kv of 7 and 1 161 kv of 2 transmission l...,1 500,maintenance,"line,lines oos,",transmission
5,1 500 KV (OF 7) AND 1 161 KV (OF 2) TRANSMISSI...,BROWNS FERRY 3,2,20071101,100.0,0.00,1 500 kv of 7 and 1 161 kv of 2 transmission l...,1 500,maintenance,"line,lines oos,",transmission
6,#1 COOLING TOWER TRANSFORMER OOS,BROWNS FERRY 1,5,20081203,87.6,0.93,1 cool tower transformer oos,1 cool,tower transformer oos,"transformer oos,",transformer
7,#1 COOLING TOWER TRANSFORMER OOS,BROWNS FERRY 2,15,20081128,100.0,0.00,1 cool tower transformer oos,1 cool,tower transformer oos,"transformer oos,",transformer
...,...,...,...,...,...,...,...,...,...,...,...
2797,YELLOW GRID RISK - 2 LINES OOS,SEQUOYAH 1,5,20121126,100.0,0.00,yellow grid risk 2 lines oos,yellow grid,yellow grid,"line,grid risk,lines oos,",transmission
2800,YELLOW GRID CONDITION DUE TO SCHEDULED GRID MA...,SEQUOYAH 1,3,20061119,100.0,0.00,yellow grid condition due to scheduled grid ma...,yellow grid,yellow grid,"grid condition,",grid issue
2801,YELLOW GRID; EAST CLEVELAND 161KV LINE OOS FOR...,SEQUOYAH 1,1,20110920,100.0,0.00,yellow grid east cleveland 161kv line oos for ...,yellow grid,yellow grid,"line oos,line,161kv,line oos,",transmission
2802,YELLOW GRID; EAST CLEVELAND 161KV LINE OOS FOR...,SEQUOYAH 2,1,20110920,100.0,0.00,yellow grid east cleveland 161kv line oos for ...,yellow grid,yellow grid,"line oos,line,161kv,line oos,",transmission


In [41]:
data1.to_csv('C:/Users/wangs/OneDrive/桌面/Power Status_TVA_issue_type_08-10-2022_type1.csv')

In [79]:
data2=data.append(data1).drop_duplicates(keep=False)

C:\Users\wangs\AppData\Local\Temp\ipykernel_7228\272746358.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data2=data.append(data1).drop_duplicates(keep=False)


In [80]:
data2 = data2.reset_index(drop=True)
data2

,Unique Issue,Plant,Consecutive days de-rated,Event date(start),Average Power(% of rate),Generation Losses(in Millions),Reason_text,first 2 words,Class,TW,term
0,0800 ON 5/11/07 TAGOUT OF 500KV BUS 1,BROWNS FERRY 2,2,20070510,100.0,0.00,0800 on 5 11 07 tagout of 500kv bus 1,0800 on,NaN,,NaN
1,0800 ON 5/11/07 TAGOUT OF 500KV BUS 1,BROWNS FERRY 3,2,20070510,100.0,0.00,0800 on 5 11 07 tagout of 500kv bus 1,0800 on,NaN,,NaN
2,#1 & 4 CIVS CLOSED,BROWNS FERRY 2,1,20020114,94.0,0.09,1 4 civs closed,1 4,close,,NaN
3,100% ELECTRICAL CAPABILITY,BROWNS FERRY 2,2,20210114,98.0,0.06,100 electrical capability,100 electrical,100% electrical capability,,NaN
4,100 MVAR INCOMING LIMIT,WATTS BAR 1,3,20161213,100.0,0.00,100 mvar incoming limit,100 mvar,incoming limit,,NaN
...,...,...,...,...,...,...,...,...,...,...,...
1773,YELLOW GRID,SEQUOYAH 1,3,20080517,100.0,0.00,yellow grid,yellow grid,yellow grid,,NaN
1774,YELLOW GRID,SEQUOYAH 2,1,20130523,100.0,0.00,yellow grid,yellow grid,yellow grid,,NaN
1775,YELLOW GRID; RACCOON MOUNTAIN PUMPING IN PROGRESS,SEQUOYAH 1,2,20110921,100.0,0.00,yellow grid raccoon mountain pumping in progress,yellow grid,yellow grid,,NaN
1776,YELLOW GRID; RACCOON MOUNTAIN PUMPING IN PROGRESS,SEQUOYAH 2,2,20110921,100.0,0.00,yellow grid raccoon mountain pumping in progress,yellow grid,yellow grid,,NaN


In [81]:
for i in range(len(data2)):
    print(i)
    issue = data2.loc[i, 'Reason_text']
    for j in range(len(filter_out)):
        filter_ = filter_out.loc[j, 'filter_out']
        if filter_ in issue:
            data2.loc[i, 'term'] = filter_

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [82]:
data2

,Unique Issue,Plant,Consecutive days de-rated,Event date(start),Average Power(% of rate),Generation Losses(in Millions),Reason_text,first 2 words,Class,TW,term
0,0800 ON 5/11/07 TAGOUT OF 500KV BUS 1,BROWNS FERRY 2,2,20070510,100.0,0.00,0800 on 5 11 07 tagout of 500kv bus 1,0800 on,NaN,,NaN
1,0800 ON 5/11/07 TAGOUT OF 500KV BUS 1,BROWNS FERRY 3,2,20070510,100.0,0.00,0800 on 5 11 07 tagout of 500kv bus 1,0800 on,NaN,,NaN
2,#1 & 4 CIVS CLOSED,BROWNS FERRY 2,1,20020114,94.0,0.09,1 4 civs closed,1 4,close,,NaN
3,100% ELECTRICAL CAPABILITY,BROWNS FERRY 2,2,20210114,98.0,0.06,100 electrical capability,100 electrical,100% electrical capability,,NaN
4,100 MVAR INCOMING LIMIT,WATTS BAR 1,3,20161213,100.0,0.00,100 mvar incoming limit,100 mvar,incoming limit,,NaN
...,...,...,...,...,...,...,...,...,...,...,...
1773,YELLOW GRID,SEQUOYAH 1,3,20080517,100.0,0.00,yellow grid,yellow grid,yellow grid,,yellow
1774,YELLOW GRID,SEQUOYAH 2,1,20130523,100.0,0.00,yellow grid,yellow grid,yellow grid,,yellow
1775,YELLOW GRID; RACCOON MOUNTAIN PUMPING IN PROGRESS,SEQUOYAH 1,2,20110921,100.0,0.00,yellow grid raccoon mountain pumping in progress,yellow grid,yellow grid,,yellow
1776,YELLOW GRID; RACCOON MOUNTAIN PUMPING IN PROGRESS,SEQUOYAH 2,2,20110921,100.0,0.00,yellow grid raccoon mountain pumping in progress,yellow grid,yellow grid,,yellow


In [83]:
data3 = data2.dropna(subset = ['term'])
data3

,Unique Issue,Plant,Consecutive days de-rated,Event date(start),Average Power(% of rate),Generation Losses(in Millions),Reason_text,first 2 words,Class,TW,term
6,161 ATHENS OOS FOR TESTING,BROWNS FERRY 1,1,20100927,100.0,0.00,161 athens oos for testing,161 athens,testing,,testing
7,161 ATHENS OOS FOR TESTING,BROWNS FERRY 2,1,20100927,100.0,0.00,161 athens oos for testing,161 athens,testing,,testing
8,161 ATHENS OOS FOR TESTING,BROWNS FERRY 3,1,20100927,100.0,0.00,161 athens oos for testing,161 athens,testing,,testing
14,24 HOUR SOAK,BROWNS FERRY 3,1,20041127,92.0,0.12,24 hour soak,24 hour,soak,,soak
15,24 HR FUEL SOAK IN PROGRESS,BROWNS FERRY 3,1,20090921,94.0,0.09,24 hr fuel soak in progress,24 hr,soak,,soak
...,...,...,...,...,...,...,...,...,...,...,...
1773,YELLOW GRID,SEQUOYAH 1,3,20080517,100.0,0.00,yellow grid,yellow grid,yellow grid,,yellow
1774,YELLOW GRID,SEQUOYAH 2,1,20130523,100.0,0.00,yellow grid,yellow grid,yellow grid,,yellow
1775,YELLOW GRID; RACCOON MOUNTAIN PUMPING IN PROGRESS,SEQUOYAH 1,2,20110921,100.0,0.00,yellow grid raccoon mountain pumping in progress,yellow grid,yellow grid,,yellow
1776,YELLOW GRID; RACCOON MOUNTAIN PUMPING IN PROGRESS,SEQUOYAH 2,2,20110921,100.0,0.00,yellow grid raccoon mountain pumping in progress,yellow grid,yellow grid,,yellow


In [84]:
data3.to_csv('C:/Users/wangs/OneDrive/桌面/Power Status_TVA_issue_type_08-10-2022_type2.csv')

In [85]:
data4=data2.append(data3).drop_duplicates(keep=False)

C:\Users\wangs\AppData\Local\Temp\ipykernel_7228\1853083354.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data4=data2.append(data3).drop_duplicates(keep=False)


In [86]:
data4 = data4.reset_index(drop=True)
data4

,Unique Issue,Plant,Consecutive days de-rated,Event date(start),Average Power(% of rate),Generation Losses(in Millions),Reason_text,first 2 words,Class,TW,term
0,0800 ON 5/11/07 TAGOUT OF 500KV BUS 1,BROWNS FERRY 2,2,20070510,100.0,0.00,0800 on 5 11 07 tagout of 500kv bus 1,0800 on,NaN,,NaN
1,0800 ON 5/11/07 TAGOUT OF 500KV BUS 1,BROWNS FERRY 3,2,20070510,100.0,0.00,0800 on 5 11 07 tagout of 500kv bus 1,0800 on,NaN,,NaN
2,#1 & 4 CIVS CLOSED,BROWNS FERRY 2,1,20020114,94.0,0.09,1 4 civs closed,1 4,close,,NaN
3,100% ELECTRICAL CAPABILITY,BROWNS FERRY 2,2,20210114,98.0,0.06,100 electrical capability,100 electrical,100% electrical capability,,NaN
4,100 MVAR INCOMING LIMIT,WATTS BAR 1,3,20161213,100.0,0.00,100 mvar incoming limit,100 mvar,incoming limit,,NaN
...,...,...,...,...,...,...,...,...,...,...,...
661,WATERBOX CLEANING,BROWNS FERRY 2,1,20130709,41.0,0.89,waterbox cleaning,waterbox cleaning,cleaning,,NaN
662,WEATHER ALERT,BROWNS FERRY 1,1,20140302,100.0,0.00,weather alert,weather alert,alert,,NaN
663,WEATHER ALERT,BROWNS FERRY 2,1,20140302,100.0,0.00,weather alert,weather alert,alert,,NaN
664,WILL CONDUCT A TURBINE VALVE FREEDOM TEST TODAY,SEQUOYAH 2,1,20120505,100.0,0.00,will conduct a turbine valve freedom test today,will conduct,NaN,,NaN


In [111]:
#maintenance, "reduce power due to…" or "reduced for", repair, shut down / shutdown, alert 
for i in range(len(data4)):
    print(i)
    issue = data4.loc[i, 'Reason_text']
    #For alert
    if 'alert' in issue:
        list_ = issue.split('alert')
        data4.loc[i, 'term'] = ' '.join(list_[:1]) + 'alert'
    #For maintenance
    if 'maintenance' in issue:
        list_ = issue.split('maintenance')
        data4.loc[i, 'term'] = ' '.join(list_[:1]) + 'alert'
    #For repair
    if 'repairs' in issue:
        if issue.split(' ').index('repairs'):
            index = issue.split(' ').index('repairs')
            if index >=2:
                data4.loc[i, 'term'] = ' '.join(issue.split(' ')[index-2:index+1])
            elif index <= 1:
                data4.loc[i, 'term'] = issue
    else:
        if 'repair' in issue:
            if issue.split(' ').index('repair'):
                index = issue.split(' ').index('repair')
                if index >=2:
                    data4.loc[i, 'term'] = ' '.join(issue.split(' ')[index-2:index+1])
                elif index <= 1:
                    data4.loc[i, 'term'] = issue
    #For due to
    if 'due to' in issue:
        list_ = issue.split('due to')
        data4.loc[i, 'term'] = list_[-1]
    #For reduced for
    if 'reduced for' in issue:
        list_ = issue.split('reduced for')
        data4.loc[i, 'term'] = list_[1:]
    #For shutdown
    if 'shut' in issue:
        list_ = issue.split('shut')
        data4.loc[i, 'term'] = list_[-1]

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [112]:
data4

,Unique Issue,Plant,Consecutive days de-rated,Event date(start),Average Power(% of rate),Generation Losses(in Millions),Reason_text,first 2 words,Class,TW,term
0,0800 ON 5/11/07 TAGOUT OF 500KV BUS 1,BROWNS FERRY 2,2,20070510,100.0,0.00,0800 on 5 11 07 tagout of 500kv bus 1,0800 on,NaN,,NaN
1,0800 ON 5/11/07 TAGOUT OF 500KV BUS 1,BROWNS FERRY 3,2,20070510,100.0,0.00,0800 on 5 11 07 tagout of 500kv bus 1,0800 on,NaN,,NaN
2,#1 & 4 CIVS CLOSED,BROWNS FERRY 2,1,20020114,94.0,0.09,1 4 civs closed,1 4,close,,NaN
3,100% ELECTRICAL CAPABILITY,BROWNS FERRY 2,2,20210114,98.0,0.06,100 electrical capability,100 electrical,100% electrical capability,,NaN
4,100 MVAR INCOMING LIMIT,WATTS BAR 1,3,20161213,100.0,0.00,100 mvar incoming limit,100 mvar,incoming limit,,NaN
...,...,...,...,...,...,...,...,...,...,...,...
661,WATERBOX CLEANING,BROWNS FERRY 2,1,20130709,41.0,0.89,waterbox cleaning,waterbox cleaning,cleaning,,NaN
662,WEATHER ALERT,BROWNS FERRY 1,1,20140302,100.0,0.00,weather alert,weather alert,alert,,weather alert
663,WEATHER ALERT,BROWNS FERRY 2,1,20140302,100.0,0.00,weather alert,weather alert,alert,,weather alert
664,WILL CONDUCT A TURBINE VALVE FREEDOM TEST TODAY,SEQUOYAH 2,1,20120505,100.0,0.00,will conduct a turbine valve freedom test today,will conduct,NaN,,NaN


In [113]:
data4.to_csv('C:/Users/wangs/OneDrive/桌面/Power Status_TVA_issue_type_08-10-2022_type3.csv')

In [41]:
code_list=df1['ID'].tolist()
df2=df2[~df2['ID'].isin(code_list)]

TypeError: unsupported operand type(s) for -: 'str' and 'str'